In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

#노트북 안에 그래프 그리기 용
%matplotlib inline

#그래프 스타일
plt.style.use('ggplot')

#그래프 마이너스 폰트 깨지는 문제
mpl.rcParams['axes.unicode_minus'] = False

In [ ]:
train = pd.read_csv("../input/bike-sharing-demand/train.csv", parse_dates=["datetime"])
train.shape

In [ ]:
#train.coluns
#train.dtypes
train.info()

데이터 타임이랑 int, float형식이 있다...

In [ ]:
train.head(20)

windspeed는 0이 많네... 바람측정 없을 경우도 0으로 넣은건가? 의심한다

In [ ]:
train.temp.describe()

In [ ]:
train.isnull().sum()

NULL인 데이터는 없다..

In [ ]:
import missingno as msno
msno.matrix(train, figsize=(12,5))

NULL인 데이터를 시각화하여 확인하는건데 역시 NULL이 없다.

In [ ]:
train["year"] = train["datetime"].dt.year
train["month"] = train["datetime"].dt.month
train["day"] = train["datetime"].dt.day
train["hour"] = train["datetime"].dt.hour
train["minute"] = train["datetime"].dt.minute
train["second"] = train["datetime"].dt.second
train.shape

In [ ]:
train.head()

In [ ]:
figure, ((ax1,ax2,ax3), (ax4,ax5,ax6)) = plt.subplots(nrows=2, ncols=3)
figure.set_size_inches(18,8)

sns.barplot(data=train, x="year", y="count", ax=ax1)
sns.barplot(data=train, x="month", y="count", ax=ax2)
sns.barplot(data=train, x="day", y="count", ax=ax3)
sns.barplot(data=train, x="hour", y="count", ax=ax4)
sns.barplot(data=train, x="minute", y="count", ax=ax5)
sns.barplot(data=train, x="second", y="count", ax=ax6)

ax1.set(ylabel='Count',title="Rental volume by year")
ax2.set(xlabel='month',title="Rental volume by month")
ax3.set(xlabel='day', title="Rental volume by day")
ax4.set(xlabel='hour', title="Rental volume by time")

연도별 : 2011 <<<< 2012  홍보효과가 큰가???
월별 : 6월이 많고 7~10도 많구나. 1월엔 추워서 적은듯?
일별 : 19일까지만 있다... 나머지는 test.csv에 있다. 이건 사용X
시간별 : 출퇴근 시간대에 대여량 많다. 주말, 평일 구분해야할듯..
분,초 : 다 0이여서 의미가 없다.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2)
fig.set_size_inches(12, 10)
sns.boxplot(data=train,y="count", orient="v", ax=axes[0][0])
sns.boxplot(data=train,y="count",x="season",orient="v",ax=axes[0][1])
sns.boxplot(data=train,y="count",x="hour",orient="v",ax=axes[1][0])
sns.boxplot(data=train,y="count",x="workingday",orient="v",ax=axes[1][1])

axes[0][0].set(ylabel='Count',title="Rental volume by time")
axes[0][1].set(xlabel='Season', ylabel='Count',title="Rental volume by season")
axes[1][0].set(xlabel='Hour Of The Day', ylabel='Count',title="Rental volume by time")
axes[1][1].set(xlabel='Working Day', ylabel='Count',title="Rental volume by working days")


대여량 : 특정구간에 모여있음..
계절별 : 봄<겨울<여름,가을
시간별 : 출퇴근시간..??
근무일여부 : 큰 차이는 없는데 휴일에 조금 더 많음

In [ ]:
train["dayofweek"] = train["datetime"].dt.dayofweek
train.shape

In [ ]:
train["dayofweek"].value_counts()

In [ ]:
fig,(ax1,ax2,ax3,ax4,ax5)=plt.subplots(nrows=5)
fig.set_size_inches(18,25)

sns.pointplot(data=train, x="hour", y="count", ax=ax1)

sns.pointplot(data=train, x="hour", y="count", hue="workingday", ax=ax2)

sns.pointplot(data=train, x="hour", y="count", hue="dayofweek", ax=ax3)

sns.pointplot(data=train, x="hour", y="count", hue="weather", ax=ax4)

sns.pointplot(data=train, x="hour", y="count", hue="season", ax=ax5)

In [ ]:
corrMatt = train[["temp", "atemp", "casual", "registered", "humidity", "windspeed", "count"]]
corrMatt = corrMatt.corr()
print(corrMatt)

mask = np.array(corrMatt)
mask[np.tril_indices_from(mask)] = False

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20,10)
sns.heatmap(corrMatt, mask=mask,vmax=.8, square=True, annot=True)

온도, 습도, 풍속은 연관관계가 없는듯
대여량과 가장 연관이 높은 건 등론된 대여자가 많은데 test에는 데이터 값이 없다.
atemp와 temp는 0.98로 상관관계가 높지만 온도와 체감온도로 피처로 사용하기에 적합하지 않은 것 같다.

In [ ]:
fig,(ax1,ax2,ax3) = plt.subplots(ncols=3)
fig.set_size_inches(12, 5)
sns.regplot(x="temp", y="count", data=train, ax=ax1)
sns.regplot(x="windspeed", y="count", data=train,ax=ax2)
sns.regplot(x="humidity", y="count", data=train,ax=ax3)

풍속 0 이 너무 많고, 습도도 0이랑 100에 일부 몰려있다.

In [ ]:
def concatenate_year_month(datetime):
    return "{0}-{1}".format(datetime.year, datetime.month)

train["year_month"] = train["datetime"].apply(concatenate_year_month)

print(train.shape)
train[["datetime", "year_month"]].head()

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)
fig.set_size_inches(18, 4)

sns.barplot(data=train, x="year", y="count", ax=ax1)
sns.barplot(data=train, x="month", y="count", ax=ax2)

fig, ax3 = plt.subplots(nrows=1, ncols=1)
fig.set_size_inches(18, 4)

sns.barplot(data=train, x="year_month", y="count", ax=ax3)

매년 증가하는 추세?!

In [ ]:
# trainWithoutOutliers
trainWithoutOutliers = train[np.abs(train["count"] - train["count"].mean()) <= (3*train["count"].std())]

print(train.shape)
print(trainWithoutOutliers.shape)

끝에 몰려있는 값을 제거.

In [ ]:
# count값의 데이터 분포도를 파악

figure, axes = plt.subplots(ncols=2, nrows=2)
figure.set_size_inches(12, 10)

sns.distplot(train["count"], ax=axes[0][0])
stats.probplot(train["count"], dist='norm', fit=True, plot=axes[0][1])
sns.distplot(np.log(trainWithoutOutliers["count"]), ax=axes[1][0])
stats.probplot(np.log1p(trainWithoutOutliers["count"]), dist='norm', fit=True, plot=axes[1][1])